Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [52]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [53]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [54]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [55]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

** Add L2 regularization for logistic regression model first. Note: L2 is a penalty to the Loss function.

In [44]:
batch_size = 128
L2_weight = 0.5e-3

graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros(num_labels))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + L2_weight * tf.nn.l2_loss(weights)
    
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [45]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if(step % 500 == 0):
            print("Minibatch loss at step: %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation dataset accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test dataset accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step: 0: 16.580879
Minibatch accuracy: 7.8%
Validation dataset accuracy: 10.7%
Minibatch loss at step: 500: 2.897196
Minibatch accuracy: 71.9%
Validation dataset accuracy: 70.7%
Minibatch loss at step: 1000: 3.501479
Minibatch accuracy: 75.0%
Validation dataset accuracy: 73.2%
Minibatch loss at step: 1500: 2.461379
Minibatch accuracy: 76.6%
Validation dataset accuracy: 74.2%
Minibatch loss at step: 2000: 2.302784
Minibatch accuracy: 82.0%
Validation dataset accuracy: 75.0%
Minibatch loss at step: 2500: 2.557680
Minibatch accuracy: 72.7%
Validation dataset accuracy: 75.7%
Minibatch loss at step: 3000: 2.381720
Minibatch accuracy: 72.7%
Validation dataset accuracy: 76.0%
Test dataset accuracy: 84.0%


**Learning rate: 0.1, batch_size: 128, steps: 3001. Test accuracy: 84.0%.

** Add L2 regularization for one hidden layer fully connected network.

In [34]:
batch_size = 128
relu_count = 1024
L2_weight = 0.5e-3

graph_nn_l2 = tf.Graph()
with graph_nn_l2.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_hidden_layer = tf.Variable(tf.truncated_normal([image_size * image_size, relu_count]))
    biases_hidden_layer = tf.Variable(tf.zeros(relu_count))
    
    weights_output_layer = tf.Variable(tf.truncated_normal([relu_count, num_labels]))
    biases_output_layer = tf.Variable(tf.zeros(num_labels))
    
    hidden_layer_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden_layer) + biases_hidden_layer)
    
    logits = tf.matmul(hidden_layer_output, weights_output_layer) + biases_output_layer
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))\
    + L2_weight * (tf.nn.l2_loss(weights_hidden_layer) + tf.nn.l2_loss(weights_output_layer))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights_hidden_layer) + biases_hidden_layer),
                  weights_output_layer) 
        + biases_output_layer)
    
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_test_dataset, weights_hidden_layer) + biases_hidden_layer),
                  weights_output_layer)
        + biases_output_layer)

In [35]:
num_steps = 3001

with tf.Session(graph=graph_nn_l2) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if(step % 500 == 0):
            print("Minibatch loss at step: %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation dataset accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test dataset accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step: 0: 471.538483
Minibatch accuracy: 12.5%
Validation dataset accuracy: 32.7%
Minibatch loss at step: 500: 142.494095
Minibatch accuracy: 84.4%
Validation dataset accuracy: 79.5%
Minibatch loss at step: 1000: 99.143898
Minibatch accuracy: 78.1%
Validation dataset accuracy: 81.2%
Minibatch loss at step: 1500: 73.990669
Minibatch accuracy: 89.1%
Validation dataset accuracy: 82.2%
Minibatch loss at step: 2000: 56.160179
Minibatch accuracy: 88.3%
Validation dataset accuracy: 82.2%
Minibatch loss at step: 2500: 43.564430
Minibatch accuracy: 86.7%
Validation dataset accuracy: 83.5%
Minibatch loss at step: 3000: 33.949070
Minibatch accuracy: 88.3%
Validation dataset accuracy: 84.4%
Test dataset accuracy: 91.6%


Test accuracy: 91.3%, so far, so good.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [37]:
num_steps = 3001
small_batch_data = train_dataset[:batch_size * 7, :]
small_batch_labels = train_labels[:batch_size * 7]
with tf.Session(graph=graph_nn_l2) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (small_batch_data.shape[0] - batch_size)
        
        batch_data = small_batch_data[offset:(offset + batch_size), :]
        batch_labels = small_batch_labels[offset:(offset + batch_size)]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if(step % 500 == 0):
            print("Minibatch loss at step: %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation dataset accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test dataset accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step: 0: 612.910034
Minibatch accuracy: 8.6%
Validation dataset accuracy: 38.6%
Minibatch loss at step: 500: 122.551140
Minibatch accuracy: 100.0%
Validation dataset accuracy: 75.2%
Minibatch loss at step: 1000: 95.439880
Minibatch accuracy: 100.0%
Validation dataset accuracy: 75.2%
Minibatch loss at step: 1500: 74.326385
Minibatch accuracy: 100.0%
Validation dataset accuracy: 75.3%
Minibatch loss at step: 2000: 57.883617
Minibatch accuracy: 100.0%
Validation dataset accuracy: 75.3%
Minibatch loss at step: 2500: 45.078438
Minibatch accuracy: 100.0%
Validation dataset accuracy: 75.3%
Minibatch loss at step: 3000: 35.106117
Minibatch accuracy: 100.0%
Validation dataset accuracy: 75.4%
Test dataset accuracy: 83.2%


Training dataset accuracy reaches 100% very quickly, and the final test dataset accuracy is, in fact, not as bad as I expect.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

** Apply drop out to one hidden layer neural network. Too see if it's better than L2 norm.

In [73]:
batch_size = 128
relu_count = 1024
L2_weight = 0.5e-3

graph_nn_dropout = tf.Graph()
with graph_nn_dropout.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_hidden_layer = tf.Variable(tf.truncated_normal([image_size * image_size, relu_count]))
    biases_hidden_layer = tf.Variable(tf.zeros(relu_count))
    
    weights_output_layer = tf.Variable(tf.truncated_normal([relu_count, num_labels]))
    biases_output_layer = tf.Variable(tf.zeros(num_labels))
    
    hidden_layer_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden_layer) + biases_hidden_layer)
    
#   apply drop out.
    dropout_probablity = tf.constant(0.5)
    output_layer_activition = tf.nn.dropout(hidden_layer_output, dropout_probablity)
    
    logits = tf.matmul(output_layer_activition, weights_output_layer) + biases_output_layer\
    + L2_weight * (tf.nn.l2_loss(weights_hidden_layer) + tf.nn.l2_loss(weights_output_layer))
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5e-2).minimize(loss)
    
    train_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_train_dataset, weights_hidden_layer) + biases_hidden_layer),
                  weights_output_layer) 
        + biases_output_layer)
    
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights_hidden_layer) + biases_hidden_layer),
                  weights_output_layer) 
        + biases_output_layer)
    
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_test_dataset, weights_hidden_layer) + biases_hidden_layer),
                  weights_output_layer)
        + biases_output_layer)

In [75]:
num_steps = 6001
with tf.Session(graph=graph_nn_dropout) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        if(step % 500 == 0):
            print("Minibatch loss at step: %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation dataset accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test dataset accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step: 0: 487.267212
Minibatch accuracy: 7.0%
Validation dataset accuracy: 10.0%
Minibatch loss at step: 500: 99.905197
Minibatch accuracy: 82.0%
Validation dataset accuracy: 78.3%
Minibatch loss at step: 1000: 97.113907
Minibatch accuracy: 77.3%
Validation dataset accuracy: 80.3%
Minibatch loss at step: 1500: 63.341541
Minibatch accuracy: 85.2%
Validation dataset accuracy: 80.9%
Minibatch loss at step: 2000: 61.027161
Minibatch accuracy: 86.7%
Validation dataset accuracy: 81.6%
Minibatch loss at step: 2500: 61.532345
Minibatch accuracy: 82.8%
Validation dataset accuracy: 81.9%
Minibatch loss at step: 3000: 51.948311
Minibatch accuracy: 80.5%
Validation dataset accuracy: 82.2%
Minibatch loss at step: 3500: 66.544884
Minibatch accuracy: 83.6%
Validation dataset accuracy: 82.8%
Minibatch loss at step: 4000: 38.978706
Minibatch accuracy: 85.2%
Validation dataset accuracy: 83.0%
Minibatch loss at step: 4500: 29.721146
Minibatch accuracy: 85.2%
Validation datase

1. Hidden layer neurons: 1024, learning rate: 0.5, num_steps: 3001. With dropout only, Test accuracy: 87.5%
2. Hidden layer neurons: 1024, learning rate: 0.5, num_steps: 3001. dropout + L2 norm, Test accuracy: 87.2%
3. Hidden layer neurons: 1024, learning rate: 0.05, num_steps: 3001. dropout + L2 norm, Test accuracy: 90.6%
4. 3. Hidden layer neurons: 1024, learning rate: 0.05, num_steps: 6001. dropout + L2 norm, Test accuracy: 90.6%

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
